In [1]:
import pandas as pd
import numpy as np
from glob import glob
from re import match, sub
from missingno import matrix as mmatrix
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from math import ceil
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from statsmodels.formula.api import ols
from os import listdir
from statsmodels.formula.api import ols

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
if 'figure_dir.txt' in listdir():
    with open('figure_dir.txt', 'r') as f:
        fig_dir = f.readline().strip().replace('\\', '/')
        tbl_dir = f"{fig_dir.rsplit('/', maxsplit=2)[0]}/Tables/"
else:
    fig_dir = 'Figures/'
    tbl_dir = 'Tables/'

def trim_trailing_zeros(number_string):
    '''
    Take a string with numbers and remove any trailing zeros from the number.
    '''
    return sub(
        r'(?:\.0+|(\.\d+)0+)(%?)$', 
        '\\1\\2', number_string
    )

def neaten_ticks(
        ax, axis, tick_range, ticks, dp=0, fmt='f', twin=False
    ):
    '''
    Take a plot ax and convert its x or y axis into a neatened range frame.
    '''
    # Set sets.
    eval(f"ax.set_{axis}ticks")(sorted([*ticks, *tick_range]))
    is_log = eval(f"ax.get_{axis}scale")() == 'log' 
    # Set labels.
    eval(f"ax.set_{axis}ticklabels")([
        f"$\\mathdefault{{10^{{{ex:.0f}}}}}$" 
        if is_log and int((ex := np.log10(e))) == ex 
        else trim_trailing_zeros(eval(f'''f"{{e:.0{dp}{fmt}}}"'''))
        for e in eval(f"ax.get_{axis}ticks")()
    ])
    # Limit axis outline to data range.
    ax.spines[
        'right' if twin else 'bottom' if axis == 'x' else 'left'
    ].set_bounds(*tick_range)

def show(
    fig, axes=[], tight=False, despine=True, despine_twin=False, 
    delax=True, file=None, legend_loc=0, filetype='pdf'
):
    '''Style and show a matplotlib plot.'''
    if tight: plt.tight_layout() # Neaten subplots.
    # Remove unnecessary outlines from plots with twin axes.
    # Note that this method assumes all axes have twins.
    if despine_twin:
        for ax in axes[::2]:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
        for ax in axes[1::2]:
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
    # Remove unnecessary outlines.
    elif despine:
        for ax in axes:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
    for ax in axes:
    # Remove unused subplots.
        if delax and not ax.has_data(): fig.delaxes(ax)
        ax.tick_params(
            axis='both', which='minor', 
            bottom=False, left=False, right=False
        )
        # Style the legend if a legend is present.
        if ax.get_legend():
            handles, labels = ax.get_legend_handles_labels()
            labels = [e.title() for e in labels]
            legend_title = ax.get_legend().get_title().get_text().title()
            ax.legend(
                handles=handles, labels=labels, 
                title=legend_title, loc=legend_loc
            )
    # Save file if a filename is provided.
    if file:
        plt.savefig(
            f'{fig_dir}/Plots/{file}.{filetype}', dpi = 300,
            bbox_inches = 'tight', pad_inches = 1/25
        )
    plt.show()
    

In [3]:
CSVs = glob('data/*/*.csv')

for csv in CSVs:
    csv = csv.replace('\\', '/')
    file = match(r'data.*/(.*?)\.csv', csv).group(1)
    print(file)
    exec(f'{file} = pd.read_csv("{csv}")')
    exec(f'{file}["state"] = "{file.split('_')[-1].upper()}"')


forecastdemand_qld
forecastdemand_sa
forecastdemand_vic
temprature_qld
temprature_sa
temprature_vic
totaldemand_qld
totaldemand_sa
totaldemand_vic
forecastdemand_nsw
temperature_nsw
totaldemand_nsw


In [4]:
def get_metadata(data, name):
    return {
        'Dataset': name, 
        'Records': len(data), 
        'Variables': len([e for e in data.columns if e != 'state']),
        'Size in memory (MB)': data.drop(columns=['state']).memory_usage(deep=True).sum() / 1e6
    }

metadata = []


In [75]:
dem = pd.concat([totaldemand_qld, totaldemand_sa, totaldemand_vic])

dem.DATETIME = pd.to_datetime(dem.DATETIME, format="%Y-%m-%d %H:%M:%S")
totaldemand_nsw.DATETIME = pd.to_datetime(totaldemand_nsw.DATETIME, format="%d/%m/%Y %H:%M")
dem = pd.concat([dem, totaldemand_nsw]).reset_index(drop=True)
# dem = dem.drop(columns=['REGIONID'])

# client['data']['total_demand'].insert_many(dem.to_dict(orient='records'))

dem = dem[dem.state == 'NSW'].reset_index(drop=True)

# print(dem.isnull().sum())

metadata.append(get_metadata(dem, 'totaldemand_nsw'))

dem


,DATETIME,TOTALDEMAND,REGIONID,state
0,2010-01-01 00:00:00,8038.00,NSW1,NSW
1,2010-01-01 00:30:00,7809.31,NSW1,NSW
2,2010-01-01 01:00:00,7483.69,NSW1,NSW
3,2010-01-01 01:30:00,7117.23,NSW1,NSW
4,2010-01-01 02:00:00,6812.03,NSW1,NSW
...,...,...,...,...
196508,2021-03-17 22:00:00,7419.77,NSW1,NSW
196509,2021-03-17 22:30:00,7417.91,NSW1,NSW
196510,2021-03-17 23:00:00,7287.32,NSW1,NSW
196511,2021-03-17 23:30:00,7172.39,NSW1,NSW


In [6]:
fore = pd.concat([
    forecastdemand_qld, forecastdemand_sa, forecastdemand_vic, forecastdemand_nsw
])

fore = fore[fore.state == 'NSW'].reset_index(drop=True)

for col in ["LASTCHANGED", "DATETIME"]:
    fore[col] = pd.to_datetime(fore[col], format="%Y-%m-%d %H:%M:%S")

metadata.append(get_metadata(fore, 'forecastdemand_nsw'))

fore = fore[fore.PERIODID.isin([2, 48])].drop_duplicates()\
    .reset_index(drop=True).drop(columns=['REGIONID'])

print(fore.isnull().sum())

# fore_piv = fore.pivot(columns='PERIODID', index='DATETIME', values='FORECASTDEMAND').reset_index()
# fore_piv.columns = [e if e=='DATETIME' else f"h{e/2:.0f}_ahead" for e in fore_piv.columns]

fore


PREDISPATCHSEQNO    0
PERIODID            0
FORECASTDEMAND      0
LASTCHANGED         0
DATETIME            0
state               0
dtype: int64


,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME,state
0,2009123041,48,7822.38,2009-12-31 00:01:34,2010-01-01 00:00:00,NSW
1,2009123139,2,7789.50,2009-12-31 23:01:24,2010-01-01 00:00:00,NSW
2,2009123042,48,7715.68,2009-12-31 00:31:25,2010-01-01 00:30:00,NSW
3,2009123140,2,7603.17,2009-12-31 23:31:32,2010-01-01 00:30:00,NSW
4,2009123043,48,7482.56,2009-12-31 01:01:17,2010-01-01 01:00:00,NSW
...,...,...,...,...,...,...
327495,2021031737,2,7316.62,2021-03-17 22:01:34,2021-03-17 23:00:00,NSW
327496,2021031640,48,7011.96,2021-03-16 23:31:34,2021-03-17 23:30:00,NSW
327497,2021031738,2,7187.72,2021-03-17 22:31:36,2021-03-17 23:30:00,NSW
327498,2021031641,48,6932.43,2021-03-17 00:01:34,2021-03-18 00:00:00,NSW


In [76]:
temp = pd.concat([temprature_qld, temperature_nsw, temprature_vic])

temp.DATETIME = pd.to_datetime(temp.DATETIME, format="%d/%m/%Y %H:%M")

temprature_sa.DATETIME = pd.to_datetime(temprature_sa.DATETIME, format="%Y-%m-%d %H:%M:%S")
temp = pd.concat([temp, temprature_sa])\
    .drop(columns = ['Unnamed: 0']).reset_index(drop=True)

# client['data']['temperature'].insert_many(temp.to_dict(orient='records'))

temp = temp[temp.state == 'NSW'].reset_index(drop=True)
metadata.append(get_metadata(temp, 'temperature_nsw'))

temp = temp.drop(columns=['LOCATION'])

print(temp.isnull().sum())

temp


DATETIME       0
TEMPERATURE    0
state          0
dtype: int64


,DATETIME,TEMPERATURE,state
0,2010-01-01 00:00:00,23.1,NSW
1,2010-01-01 00:01:00,23.1,NSW
2,2010-01-01 00:30:00,22.9,NSW
3,2010-01-01 00:50:00,22.7,NSW
4,2010-01-01 01:00:00,22.6,NSW
...,...,...,...
220321,2021-03-17 23:00:00,19.1,NSW
220322,2021-03-17 23:20:00,19.0,NSW
220323,2021-03-17 23:30:00,18.8,NSW
220324,2021-03-17 23:34:00,18.8,NSW


In [8]:
mdata = pd.DataFrame(metadata)

mdata = mdata.style.hide()\
    .format("{:,.02f}", subset=['Size in memory (MB)'])\
    .format("{:,}", subset=['Records'])\
    .format(escape="latex", subset=['Dataset'])
display(mdata)

mdata_tex = mdata.to_latex(
        label='metadata', position_float="centering", hrules=True, position='H',
        caption = "Metadata of the datasets being used. Size in memory refers to the space used after formatting, where each value is represented by an appropriate Python object (e.g. datetime values are in Timestamp format)."
)
mdata_tex = sub(r'(?s)(^.*)(\\caption{.*?}\n\\label{.*?}\n)(.*)(\\end{table})', r'\1\3\2\4', mdata_tex)
print(mdata_tex)

with open(f"{tbl_dir}/metadata.tex", 'w') as f:
    f.write(mdata_tex)


Dataset,Records,Variables,Size in memory (MB)
totaldemand\_nsw,"196,513",3,13.56
forecastdemand\_nsw,"10,906,019",6,"1,014.26"
temperature\_nsw,"220,326",3,16.30


\begin{table}[H]
\centering
\begin{tabular}{lrrr}
\toprule
Dataset & Records & Variables & Size in memory (MB) \\
\midrule
totaldemand\_nsw & 196,513 & 3 & 13.56 \\
forecastdemand\_nsw & 10,906,019 & 6 & 1,014.26 \\
temperature\_nsw & 220,326 & 3 & 16.30 \\
\bottomrule
\end{tabular}
\caption{Metadata of the datasets being used. Size in memory refers to the space used after formatting, where each value is represented by an appropriate Python object (e.g. datetime values are in Timestamp format).}
\label{metadata}
\end{table}



In [56]:
df = pd.merge(temp, dem, on=['DATETIME', 'state'], how='outer')\
    .dropna()

# ohe = OneHotEncoder(sparse_output=False)
# df[sorted(df.state.unique())] = ohe.fit_transform(df.state.to_numpy().reshape(-1, 1))
# df = df.drop(columns=['state'])
df['hour'] = df.DATETIME.dt.strftime('%H').astype(int)
df['day'] = df.DATETIME.dt.strftime('%w').astype(int)
df['month'] = df.DATETIME.dt.strftime('%m').astype(int)
df['year'] = df.DATETIME.dt.strftime('%Y').astype(int)
df['day_label'] = df.DATETIME.dt.strftime('%a')
df['is_weekday'] = df.day.between(1, 5)
ohe_day = OneHotEncoder(sparse_output=False)
df[sorted(df.day_label.unique())] = ohe_day.fit_transform(df.day_label.to_numpy().reshape(-1, 1))

h1_ahead = df.copy()[['DATETIME', 'TOTALDEMAND']].rename(columns={'TOTALDEMAND': 'h1_ahead'})
h1_ahead['h1_hour'] = h1_ahead.DATETIME.dt.strftime('%H').astype(int)
h1_ahead['h1_day'] = h1_ahead.DATETIME.dt.strftime('%w').astype(int)
h1_ahead['h1_month'] = h1_ahead.DATETIME.dt.strftime('%m').astype(int)
h1_ahead['h1_year'] = h1_ahead.DATETIME.dt.strftime('%Y').astype(int)
h1_ahead.DATETIME -= timedelta(hours=1)
h24_ahead = df.copy()[['DATETIME', 'TOTALDEMAND']].rename(columns={'TOTALDEMAND': 'h24_ahead'})
h24_ahead['h24_hour'] = h24_ahead.DATETIME.dt.strftime('%H').astype(int)
h24_ahead['h24_day'] = h24_ahead.DATETIME.dt.strftime('%w').astype(int)
h24_ahead['h24_month'] = h24_ahead.DATETIME.dt.strftime('%m').astype(int)
h24_ahead['h24_year'] = h24_ahead.DATETIME.dt.strftime('%Y').astype(int)
h24_ahead.DATETIME -= timedelta(days=1)

for ahead in [h1_ahead, h24_ahead]:
    df = pd.merge(df, ahead, how='left', on='DATETIME')

df['date'] = df.DATETIME.dt.floor('1D')

df


,DATETIME,TEMPERATURE,state,TOTALDEMAND,REGIONID,hour,day,month,year,day_label,...,h1_hour,h1_day,h1_month,h1_year,h24_ahead,h24_hour,h24_day,h24_month,h24_year,date
0,2010-01-01 00:00:00,23.1,NSW,8038.00,NSW1,0,5,1,2010,Fri,...,1.0,5.0,1.0,2010.0,7574.85,0.0,6.0,1.0,2010.0,2010-01-01
1,2010-01-01 00:30:00,22.9,NSW,7809.31,NSW1,0,5,1,2010,Fri,...,1.0,5.0,1.0,2010.0,7343.30,0.0,6.0,1.0,2010.0,2010-01-01
2,2010-01-01 01:00:00,22.6,NSW,7483.69,NSW1,1,5,1,2010,Fri,...,2.0,5.0,1.0,2010.0,7099.73,1.0,6.0,1.0,2010.0,2010-01-01
3,2010-01-01 01:30:00,22.5,NSW,7117.23,NSW1,1,5,1,2010,Fri,...,2.0,5.0,1.0,2010.0,6779.80,1.0,6.0,1.0,2010.0,2010-01-01
4,2010-01-01 02:00:00,22.5,NSW,6812.03,NSW1,2,5,1,2010,Fri,...,3.0,5.0,1.0,2010.0,6497.47,2.0,6.0,1.0,2010.0,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195969,2021-03-17 22:00:00,19.7,NSW,7419.77,NSW1,22,3,3,2021,Wed,...,23.0,3.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17
195970,2021-03-17 22:30:00,19.5,NSW,7417.91,NSW1,22,3,3,2021,Wed,...,23.0,3.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17
195971,2021-03-17 23:00:00,19.1,NSW,7287.32,NSW1,23,3,3,2021,Wed,...,0.0,4.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17
195972,2021-03-17 23:30:00,18.8,NSW,7172.39,NSW1,23,3,3,2021,Wed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-17


In [57]:
se = pd.read_csv('data/Solar Exposure.csv').rename(columns={
    'Daily global solar exposure (MJ/m*m)': 'solar_exposure'
})

se['date'] = se.apply(lambda d: datetime(d['Year'], d['Month'], d['Day']), axis=1)

se = se[['date', 'solar_exposure']]

df = pd.merge(df, se, how='left', on='date')

se


,date,solar_exposure
0,1990-01-01,27.0
1,1990-01-02,28.6
2,1990-01-03,NaN
3,1990-01-04,9.3
4,1990-01-05,23.5
...,...,...
12499,2024-03-22,12.9
12500,2024-03-23,16.9
12501,2024-03-24,17.6
12502,2024-03-25,20.4


In [59]:
df


,DATETIME,TEMPERATURE,state,TOTALDEMAND,REGIONID,hour,day,month,year,day_label,...,h1_day,h1_month,h1_year,h24_ahead,h24_hour,h24_day,h24_month,h24_year,date,solar_exposure
0,2010-01-01 00:00:00,23.1,NSW,8038.00,NSW1,0,5,1,2010,Fri,...,5.0,1.0,2010.0,7574.85,0.0,6.0,1.0,2010.0,2010-01-01,14.6
1,2010-01-01 00:30:00,22.9,NSW,7809.31,NSW1,0,5,1,2010,Fri,...,5.0,1.0,2010.0,7343.30,0.0,6.0,1.0,2010.0,2010-01-01,14.6
2,2010-01-01 01:00:00,22.6,NSW,7483.69,NSW1,1,5,1,2010,Fri,...,5.0,1.0,2010.0,7099.73,1.0,6.0,1.0,2010.0,2010-01-01,14.6
3,2010-01-01 01:30:00,22.5,NSW,7117.23,NSW1,1,5,1,2010,Fri,...,5.0,1.0,2010.0,6779.80,1.0,6.0,1.0,2010.0,2010-01-01,14.6
4,2010-01-01 02:00:00,22.5,NSW,6812.03,NSW1,2,5,1,2010,Fri,...,5.0,1.0,2010.0,6497.47,2.0,6.0,1.0,2010.0,2010-01-01,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195969,2021-03-17 22:00:00,19.7,NSW,7419.77,NSW1,22,3,3,2021,Wed,...,3.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17,11.3
195970,2021-03-17 22:30:00,19.5,NSW,7417.91,NSW1,22,3,3,2021,Wed,...,3.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17,11.3
195971,2021-03-17 23:00:00,19.1,NSW,7287.32,NSW1,23,3,3,2021,Wed,...,4.0,3.0,2021.0,NaN,NaN,NaN,NaN,NaN,2021-03-17,11.3
195972,2021-03-17 23:30:00,18.8,NSW,7172.39,NSW1,23,3,3,2021,Wed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-17,11.3


In [60]:
df = df.drop(columns=['day', 'date']).dropna().drop_duplicates().reset_index(drop=True)

df.to_csv('data/modelling_data.csv', lineterminator='\n', index=False)


In [61]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
from getpass import getpass


In [62]:
uri = "mongodb+srv://<username>:<password>@project-data.fyzivf2.mongodb.net/?retryWrites=true&w=majority&appName=project-data"\
    .replace('<username>', quote_plus(input())).replace('<password>', quote_plus(getpass()))

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
client.admin.command('ping')
print("Pinged your deployment. You successfully connected to MongoDB!")



 doug
 ········


Pinged your deployment. You successfully connected to MongoDB!


In [79]:
print(client.list_database_names())

['data', 'admin', 'local']
